In [1]:
import tensorflow as tf
from tensorflow import keras as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,Flatten,Softmax,Conv1D,Conv2D,Embedding,MaxPool1D,MaxPool2D,Reshape
import numpy as np

tf.__version__, K.__version__

('1.9.0', '2.1.6-tf')

# 数据生成

In [2]:
# Text-CNN Parameter
embedding_size = 2 # n-gram
sequence_length = 3
num_classes = 2 # 0 or 1
filter_sizes = [2,2,2] # n-gram window
num_filters = 3

# 3 words sentences (=sequence_length is 3)
sentences = ["i love you","he loves me", "she likes baseball", "i hate you","sorry for that", "this is awful"]
labels = [1,1,1,0,0,0] # 1 is good, 0 is not good.

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
vocab_size = len(word_dict)

inputs = []
for sen in sentences:
    inputs.append([word_dict[n] for n in sen.split()])
inputs = np.array(inputs)

# outputs = np.array(labels)
outputs = []
for out in labels:
    outputs.append(np.eye(num_classes)[out]) # ONE-HOT : To using Tensor Softmax Loss function
outputs = np.array(outputs)

In [3]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=sequence_length))
model.add(Reshape((sequence_length, embedding_size, 1)))
model.add(Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_size), strides=(1,1), kernel_initializer='uniform', activation='relu'))
model.add(MaxPool2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1)))
model.add(Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_size), strides=(1,1), kernel_initializer='uniform', activation='relu'))
model.add(MaxPool2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1)))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

model.summary()
model.compile(optimizer=K.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['acc'])

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'conv2d_1/Conv2D' (op: 'Conv2D') with input shapes: [?,1,1,3], [2,2,3,3].

In [ ]:
outputs

In [ ]:
history = model.fit(inputs, outputs,
                    epochs=5000,
                    batch_size=6)

In [ ]:
# Test
test_text = 'sorry hate you'
tests = []
tests.append(np.asarray([word_dict[n] for n in test_text.split()]))

tests = np.reshape(tests, (-1,3))
predict = model.predict(tests)
result = np.argmax(predict)
if result == 0:
    print(test_text,"is Bad Mean...")
else:
    print(test_text,"is Good Mean!!")